In [ ]:
!pip install datasets bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import random
import numpy as np
import torch
import os
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, auc, precision_recall_curve
from datasets import Dataset
from transformers import (
    TrainingArguments, AutoConfig, AutoModelForSequenceClassification, AutoTokenizer
)
from peft import LoraConfig, TaskType, get_peft_model, prepare_model_for_kbit_training
from transformers import Trainer

In [ ]:
# Set seed for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)

set_seed(42)

In [ ]:
train_data = pd.read_pickle("aggregated_train_df_cleaned.pkl")
val_data = pd.read_pickle("aggregated_val_df_cleaned.pkl")
test_data = pd.read_pickle("aggregated_test_df_cleaned.pkl")

In [ ]:
def preprocess_patient_data(row):
    gender = "male" if row["gender"] == "M" else "female"
    age = f"{row['age']} years old"
    diagnoses = "none" if not row["diagnoses"] else ", ".join(row["diagnoses"])
    abnormal_labs = "none" if not row["abnormal_labs"] else ", ".join(row["abnormal_labs"])
    formatted_history = (
        f"The patient is a {age} {gender}. Diagnoses include {diagnoses}. Abnormal lab tests include {abnormal_labs}."
    )
    return formatted_history

In [ ]:
train_data["formatted_history"] = train_data.apply(preprocess_patient_data, axis=1)
val_data["formatted_history"] = val_data.apply(preprocess_patient_data, axis=1)
test_data["formatted_history"] = test_data.apply(preprocess_patient_data, axis=1)

# Combine training and validation data for k-fold cross-validation
data = pd.concat([train_data, val_data], ignore_index=True)
data["labels"] = data["is_ckd"].astype(int)

In [ ]:
train_data['formatted_history'][0]

'The patient is a 52 years old female. Diagnoses include Asymptomatic human immunodeficiency virus [HIV] infection status, Other ascites, Cachexia, Other iatrogenic hypotension, Hyposmolality and/or hyponatremia, Other dependence on machines, supplemental oxygen, Personal history of tobacco use, Tobacco use disorder, Unspecified viral hepatitis C without hepatic coma, Bipolar disorder, unspecified, Portal hypertension, Chronic airway obstruction, not elsewhere classified, Person on outside of car injured in collision with other nonmotor vehicle in nontraffic accident, Do not resuscitate status, Thrombocytopenia, unspecified, Chronic hepatitis C with hepatic coma, Posttraumatic stress disorder, Hyperpotassemia, Cirrhosis of liver without mention of alcohol, Unspecified viral hepatitis C with hepatic coma. Abnormal lab tests include Hemoglobin, Albumin, Creatinine, Bicarbonate, Sodium, Potassium, Phosphate, Chloride.'

In [ ]:
model_id = "unsloth/Llama-3.2-1B-bnb-4bit"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

def tokenize_texts(texts):
    return tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt"
    )


In [ ]:
# Metrics computation
def compute_metrics(pred):
    logits, labels = pred
    predictions = np.argmax(logits, axis=1)
    probabilities = logits[:, 1]
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    roc_auc = roc_auc_score(labels, probabilities)
    precision_curve, recall_curve, _ = precision_recall_curve(labels, probabilities)
    pr_auc = auc(recall_curve, precision_curve)
    return {"accuracy": acc, "roc_auc": roc_auc, "pr_auc": pr_auc}

In [ ]:
k = 3
kf = KFold(n_splits=k, shuffle=True, random_state=42)
fold_results = []
best_model_path = "./best_model"
best_roc_auc = 0

In [ ]:
for fold, (train_idx, val_idx) in enumerate(kf.split(data)):
    print(f"Starting fold {fold + 1}/{k}...")

    # Split the data
    train_fold = data.iloc[train_idx]
    val_fold = data.iloc[val_idx]

    # Tokenize
    train_encodings = tokenize_texts(train_fold["formatted_history"].tolist())
    val_encodings = tokenize_texts(val_fold["formatted_history"].tolist())

    train_dataset = Dataset.from_dict({
        'input_ids': train_encodings['input_ids'],
        'attention_mask': train_encodings['attention_mask'],
        'labels': train_fold["labels"].tolist()
    })

    val_dataset = Dataset.from_dict({
        'input_ids': val_encodings['input_ids'],
        'attention_mask': val_encodings['attention_mask'],
        'labels': val_fold["labels"].tolist()
    })

    # Class weights
    class_weights = compute_class_weight(
        class_weight='balanced',
        classes=np.unique(train_fold["labels"]),
        y=train_fold["labels"].tolist()
    )
    class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to('cuda' if torch.cuda.is_available() else 'cpu')

    # Model and Trainer initialization
    config = AutoConfig.from_pretrained(model_id, num_labels=2)
    model = AutoModelForSequenceClassification.from_pretrained(model_id, config=config, device_map='auto')
    model = prepare_model_for_kbit_training(model)
    lora_config = LoraConfig(
        r=16, lora_alpha=8, target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj'], lora_dropout=0.05, bias='none', task_type='SEQ_CLS'
    )
    model = get_peft_model(model, lora_config)

    model.config.pad_token_id = tokenizer.pad_token_id
    model.config.use_cache = False
    model.config.pretraining_tp = 1

    training_args = TrainingArguments(
        output_dir=f"./results_fold_{fold}",
        learning_rate=2e-4,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=1,  # Train for one epoch
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="roc_auc",
        greater_is_better=True,
        fp16=True,
        seed=42,
        report_to = "none"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    # Train and evaluate
    trainer.train()
    results = trainer.evaluate(val_dataset)
    print(f"Results for fold {fold + 1}: {results}")
    fold_results.append(results)

    # Save the best model based on ROC AUC
    if results.get("eval_roc_auc", 0) > best_roc_auc:
        best_roc_auc = results.get("eval_roc_auc", 0)
        trainer.save_model(best_model_path)
        print(f"New best model saved with ROC AUC: {best_roc_auc}")

Starting fold 1/3...


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at unsloth/Llama-3.2-1B-bnb-4bit and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-20-727dfe1332ac>:62: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant paramet

Results for fold 1: {'eval_loss': 0.26485970616340637, 'eval_accuracy': 0.8941680421284288, 'eval_roc_auc': 0.8361215410217654, 'eval_pr_auc': 0.4054642081251207, 'eval_runtime': 327.4957, 'eval_samples_per_second': 77.699, 'eval_steps_per_second': 4.858, 'epoch': 1.0}
New best model saved with ROC AUC: 0.8361215410217654
Starting fold 2/3...


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at unsloth/Llama-3.2-1B-bnb-4bit and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-20-727dfe1332ac>:62: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant paramet

Results for fold 2: {'eval_loss': 0.26560261845588684, 'eval_accuracy': 0.8980193350624852, 'eval_roc_auc': 0.8302913623212439, 'eval_pr_auc': 0.41734453533938715, 'eval_runtime': 326.4916, 'eval_samples_per_second': 77.938, 'eval_steps_per_second': 4.873, 'epoch': 1.0}
Starting fold 3/3...


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at unsloth/Llama-3.2-1B-bnb-4bit and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-20-727dfe1332ac>:62: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant paramet

Results for fold 3: {'eval_loss': 0.26993614435195923, 'eval_accuracy': 0.8924739634505797, 'eval_roc_auc': 0.8292956318499671, 'eval_pr_auc': 0.3849572441818535, 'eval_runtime': 326.2618, 'eval_samples_per_second': 77.99, 'eval_steps_per_second': 4.876, 'epoch': 1.0}


In [ ]:
avg_results = {metric: np.mean([result[metric] for result in fold_results]) for metric in fold_results[0].keys()}
std_results = {metric: np.std([result[metric] for result in fold_results]) for metric in fold_results[0].keys()}
print("Average Results Across Folds:", avg_results)
print("Standard Deviation Across Folds:", std_results)

Average Results Across Folds: {'eval_loss': 0.26679948965708417, 'eval_accuracy': 0.8948871135471647, 'eval_roc_auc': 0.8319028450643255, 'eval_pr_auc': 0.4025886625487871, 'eval_runtime': 326.7497, 'eval_samples_per_second': 77.87566666666667, 'eval_steps_per_second': 4.869, 'epoch': 1.0}
Standard Deviation Across Folds: {'eval_loss': 0.002238590579619397, 'eval_accuracy': 0.0023202850913144023, 'eval_roc_auc': 0.0030106385235081717, 'eval_pr_auc': 0.013377486781950557, 'eval_runtime': 0.5357791895921311, 'eval_samples_per_second': 0.12671314936588926, 'eval_steps_per_second': 0.00787400787401211, 'epoch': 0.0}


In [ ]:
from peft import PeftModel

base_model = AutoModelForSequenceClassification.from_pretrained(model_id, config=config, device_map='auto')
model = PeftModel.from_pretrained(base_model, best_model_path)

model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False
model.config.pretraining_tp = 1


test_encodings = tokenize_texts(test_data["formatted_history"].tolist())
test_dataset = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask'],
    'labels': test_data["is_ckd"].tolist()
})
trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
test_results = trainer.evaluate(test_dataset)
print("Test Results:", test_results)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at unsloth/Llama-3.2-1B-bnb-4bit and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-25-4368a88f022c>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Test Results: {'eval_loss': 0.26166030764579773, 'eval_model_preparation_time': 0.0032, 'eval_accuracy': 0.8974910926365796, 'eval_roc_auc': 0.8316993430266539, 'eval_pr_auc': 0.3918603634379694, 'eval_runtime': 147.4921, 'eval_samples_per_second': 91.34, 'eval_steps_per_second': 5.709}
